# 放盒子（boxing）

问题描述：


在一个二维平面上，我们要把N个给定坐标的物品放到一排的N个盒子里。

盒子的位置是固定的，坐标 $(x_i, y_i)$. 

需要满足的约束：

1. 每个盒子只能包含一个物品；
2. 每个物品都放在一个盒子里；
3. 物体到盒子的总距离之和是最小的。

要求输出物品及其对应的盒子。

-------------

在后续，我们要添加一些新的约束，观察成本的变化。比如：


- 物品1的盒子必须位于物品2盒子的左边。
- 物品5必须位于物品6所在的盒子的边上（不指定是左是右）


$$\min \sum \sum c_{ij} x_{ij} \\ 
\text{s.t.} \begin{aligned}\begin{equation*}
\begin{cases}
\sum_j x_{ij} = 1, \hspace{5pt} \forall i \\ 
\sum_i x_{ij} = 1, \hspace{5pt} \forall j \\ 
\end{cases}
\end{equation*}\end{aligned}$$



- 这一部分你会继续锻炼到：

1. 如何在模型中描述 if-else 约束；

In [2]:
from math import sqrt

N = 15
box_range = range(1, N+1)
obj_range = range(1, N+1)

import random

o_xmax = N*10
o_ymax = 2*N
box_coords = {b: (10*b, 1) for b in box_range}

obj_coords= {1: (140, 6), 2: (146, 8), 3: (132, 14), 4: (53, 28), 
             5: (146, 4), 6: (137, 13), 7: (95, 12), 8: (68, 9), 9: (102, 18), 
             10: (116, 8), 11: (19, 29), 12: (89, 15), 13: (141, 4), 14: (29, 4), 15: (4, 28)}

# the distance matrix from box i to object j
# actually we compute the square of distance to keep integer
# this does not change the essence of the problem
distances = {}
for o in obj_range:
    for b in box_range:
        dx = obj_coords[o][0]-box_coords[b][0]
        dy = obj_coords[o][1]-box_coords[b][1]
        d2 = dx*dx + dy*dy
        distances[b, o] = d2
        

{(1, 1): 16925, (2, 1): 14425, (3, 1): 12125, (4, 1): 10025, (5, 1): 8125, (6, 1): 6425, (7, 1): 4925, (8, 1): 3625, (9, 1): 2525, (10, 1): 1625, (11, 1): 925, (12, 1): 425, (13, 1): 125, (14, 1): 25, (15, 1): 125, (1, 2): 18545, (2, 2): 15925, (3, 2): 13505, (4, 2): 11285, (5, 2): 9265, (6, 2): 7445, (7, 2): 5825, (8, 2): 4405, (9, 2): 3185, (10, 2): 2165, (11, 2): 1345, (12, 2): 725, (13, 2): 305, (14, 2): 85, (15, 2): 65, (1, 3): 15053, (2, 3): 12713, (3, 3): 10573, (4, 3): 8633, (5, 3): 6893, (6, 3): 5353, (7, 3): 4013, (8, 3): 2873, (9, 3): 1933, (10, 3): 1193, (11, 3): 653, (12, 3): 313, (13, 3): 173, (14, 3): 233, (15, 3): 493, (1, 4): 2578, (2, 4): 1818, (3, 4): 1258, (4, 4): 898, (5, 4): 738, (6, 4): 778, (7, 4): 1018, (8, 4): 1458, (9, 4): 2098, (10, 4): 2938, (11, 4): 3978, (12, 4): 5218, (13, 4): 6658, (14, 4): 8298, (15, 4): 10138, (1, 5): 18505, (2, 5): 15885, (3, 5): 13465, (4, 5): 11245, (5, 5): 9225, (6, 5): 7405, (7, 5): 5785, (8, 5): 4365, (9, 5): 3145, (10, 5): 2125

In [7]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Box assignment")

bin_vars = model.addVars(N, N, vtype = GRB.BINARY, name = "bin")

model.addConstrs((gp.quicksum(bin_vars[i, j] for i in range(N)) == 1 for j in range(N)))
model.addConstrs((gp.quicksum(bin_vars[i, j] for j in range(N)) == 1 for i in range(N)))


# ============ 新增约束1 ============== 
model.addConstr(bin_vars[0, 1] == 0)
model.addConstrs(bin_vars[k - 1, 0] >= bin_vars[k, 1] for k in range(1, N))
# ============ 新增约束1 ==============

# ============ 新增约束2 ==============

# forall k in 2..N-1 then we can use the sum on the right hand side
model.addConstrs(bin_vars[k,5] <= bin_vars[k-1,4] + bin_vars[k+1,4] for k in range(1,N - 1))
    
# if 6 is in box 1 then 5 must be in 2
model.addConstr(bin_vars[0,5] <= bin_vars[1,4])

# if 6 is last, then 5 must be before last
model.addConstr(bin_vars[N - 1, 5] <= bin_vars[N - 2,4])
 
# ============ 新增约束2 ==============

model.setObjective(gp.quicksum(distances[i + 1, j + 1] * bin_vars[i, j] for i in range(N) for j in range(N)))
model.update()

model.optimize()

print(model.ObjVal)



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 225 columns and 522 nonzeros
Model fingerprint: 0x3638fabc
Variable types: 0 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 43518.000000
Presolve removed 16 rows and 16 columns
Presolve time: 0.00s
Presolved: 44 rows, 209 columns, 475 nonzeros
Variable types: 0 continuous, 209 integer (209 binary)

Root relaxation: objective 9.068000e+03, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9068.00000    0    8 43518.0000 9068.00000  79.2%     -    0s
H   

In [8]:
for i in range(N):
    for j in range(N):
        if bin_vars[i, j].x > 0:
            print(f"盒子 {i + 1 } 里放的是第 {j + 1}个物品。")


盒子 1 里放的是第 15个物品。
盒子 2 里放的是第 11个物品。
盒子 3 里放的是第 14个物品。
盒子 4 里放的是第 4个物品。
盒子 5 里放的是第 8个物品。
盒子 6 里放的是第 12个物品。
盒子 7 里放的是第 7个物品。
盒子 8 里放的是第 9个物品。
盒子 9 里放的是第 10个物品。
盒子 10 里放的是第 3个物品。
盒子 11 里放的是第 13个物品。
盒子 12 里放的是第 6个物品。
盒子 13 里放的是第 5个物品。
盒子 14 里放的是第 1个物品。
盒子 15 里放的是第 2个物品。
